A second order cone (Ice-cream cone, Lorentz cone) of dimension $n$ is the set $$K_{soc} := \{(t,x) \in \mathbb{R}^n : t \geq \|x\|_2 \}$$

So it can be used to model the epigraph of the 2-norm $\|\cdot\|_2$

In [ ]:
using JuMP
using LinearAlgebra
# Let's try out some solver for SOCP
# You will need to upgrade JuMP to >= 1.20 to use this
#  Pkg.update() should do the trick.  Then restart the kernel.
using SCS, ECOS, HiGHS, Gurobi, Ipopt

# This will only work with SCS, ECOS, and Gurobi

model = Model(Gurobi.Optimizer)
set_silent(model)
@variable(model, x[1:3])
@variable(model, t)
@constraint(model, sum(x) == 1)
@constraint(model, [t; x] in SecondOrderCone())
@objective(model, Min, t)
optimize!(model)
@assert is_solved_and_feasible(model)
value(t), value.(x)


A rotated second order cone is a conic set of the form 
$$K_{rsoc} := \{(t,u,x) \in \mathbb{R}^n : 2tu \geq \|x\|_2^2 \}$$

When $u=0.5$ it models the epigraph of the sum of squares of a vector
(but it is more general)

In [ ]:
data = [1.0, 2.0, 3.0, 4.0]
target = [0.45, 1.04, 1.51, 1.97]
model = Model(Gurobi.Optimizer)
set_silent(model)
@variable(model, θ)
@variable(model, t)
@expression(model, residuals, θ * data .- target)
@constraint(model, [t; 0.5; residuals] in RotatedSecondOrderCone())
@objective(model, Min, t)
optimize!(model)
@assert is_solved_and_feasible(model)
value(θ), value(t)

Rotated second order cones can be useful for modeling the epigraph of rational powers.  
For example, one can show that 
$$2ut \geq x^2 , 2 \frac{1}{8} x \geq u^2 \Rightarrow t \geq x^{3/2}$$

(Square the first inequality, then multiply the two together)

For example, let solve: 
$$\min x^{3/2} + y^2 : x + y = 2$$

In [ ]:
model = Model(Gurobi.Optimizer)
set_silent(model)

@variable(model, x)
@variable(model, y)

@variable(model, u >= 0)
@variable(model, t1 >= 0)
@variable(model, t2 >= 0)

@constraint(model, x+y == 2)

@constraint(model, [u, t1, x] in RotatedSecondOrderCone())
@constraint(model, [0.125, x, u] in RotatedSecondOrderCone())

@constraint(model, [t2, 0.5, y] in RotatedSecondOrderCone())

@objective(model, Min, t1 + t2)
optimize!(model)
@assert is_solved_and_feasible(model)
println(objective_value(model), ":", value(x), ",", value(y))